# ECIES : Elliptic Curve Integrated Encryption Scheme

### Reference
- [(Youtube)Elliptic Curve Integrated Encryption Scheme (ECIES) Explained](https://www.youtube.com/watch?v=O4bRUjhdFMc&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=20)
- [(Youtube)Elliptic Curve Integrated Encryption Scheme (ECIES) in Python From Scratch](https://www.youtube.com/watch?v=0hTFoVOeJi0&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=21)
- [Elliptic Curve Integrated Encryption Scheme in Python From Scratch](https://sefiks.com/2023/04/27/elliptic-curve-integrated-encryption-scheme-in-python/#google_vignette)

In [126]:
import random
import hashlib
from Crypto.Cipher import AES
import hmac

In [127]:
# y^2 = x^3 + a* x + b

In [128]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
     
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
     
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
     
    is_on_curve((x3, y3), p)
         
    return x3, y3
 
def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
     
def apply_double_and_add_method(G, k, p):
    target_point = G
     
    k_binary = bin(k)[2:] #0b1111111001
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
         
        # doubling - always
        target_point = add_points(target_point, target_point, p)
         
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
     
    is_on_curve(target_point, p)
     
    return target_point


In [129]:
# from :https://sefiks.com/2018/02/16/elegant-signatures-with-elliptic-curve-cryptography/
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)


#finite field
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [130]:
point_2G = add_points(G,G,p)

In [131]:
point_2G

(89565891926547004231252920425935692360644145829622209833684329913297188986597,
 12158399299693830322967808612713398636155367887041628176798871954788371653930)

In [132]:
tmp_point = G
for i in range(2,21):
    tmp_point = add_points(tmp_point, G, p)
    print(f"{i}G = {tmp_point}")

2G = (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
3G = (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
4G = (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018)
5G = (21505829891763648114329055987619236494102133314575206970830385799158076338148, 98003708678762621233683240503080860129026887322874138805529884920309963580118)
6G = (115780575977492633039504758427830329241728645270042306223540962614150928364886, 78735063515800386211891312544505775871260717697865196436804966483607426560663)
7G = (41948375291644419605210209193538855353224492619856392092318293986323063962044, 48361766907851246668144012348516735800090617714386977531302791340517493990618)
8G = (2126205

In [133]:
apply_double_and_add_method(G= G, k = 20, p= p)

(34773495056115281091786765947597603724784643419904767525769502836017890139287,
 8470533044743364938367028725608288731153024648869546164814808839694950063162)

In [134]:
# Q  = k(private. key) x G(public basepoint)

## Alice generates her private and public key

In [135]:
# alice private key
ka = random.getrandbits(256)

# alice public key
Q = apply_double_and_add_method(G = G, k = ka, p = p)
 

## Bob generates a random key

In [136]:
 
# bob private key
rb = random.getrandbits(256)

# bob public key - send this point to Alice
U = apply_double_and_add_method(G = G, k = rb, p = p)


# private - keeps secret
T = apply_double_and_add_method(G = Q, k = rb, p = p)

## Key Derivation Function (Public)

In [137]:
def derive_keys(T):
    tx, ty = T          
    # get x coordinate of point T as binary
    tx_binary = bin(tx)[2:]     
    
    # get its first 192-bit value
    tx_binary_cropped = tx_binary[0:192]
    
    tx_restored = int(tx_binary_cropped, 2)
    
    #sha-256
    hash_hex = hashlib.sha256(str.encode(str(tx_restored))).hexdigest()
    hash_binary = bin(int(hash_hex, 16))[2:]
    
    k1 = int(hash_binary[0:128], 2).to_bytes(16, byteorder="big")
    k2 = int(hash_binary[128:], 2).to_bytes(16, byteorder="big")
    
    return k1, k2


def find_mac(message, key):
    return hmac.new(key, message, hashlib.sha256).hexdigest()
    

In [138]:
k1, k2 = derive_keys(T)

## Encryption

In [139]:
msg = b"attack tomorrow!"

In [140]:
# obj_bob = AES.new(k1)
obj_bob = AES.new(k1, AES.MODE_CBC)

In [141]:
c = obj_bob.encrypt(msg)

In [142]:
c

b'\xe7\xa4& \x16\xc7\xa7\x98V\xd4\xec\xbbX\xfb"O'

In [143]:
r = find_mac(message= c, key =k2)

In [144]:
(U, c, r)

((90161983174431744090180783126904171570642332913795555075462404763902809959061,
  17380741816161918889507570362273197106123741449171783156190595349746560784322),
 b'\xe7\xa4& \x16\xc7\xa7\x98V\xd4\xec\xbbX\xfb"O',
 '99220ace9ca39617c92b1798de8609588bbf2ff72163453d4c681c54bece8cd6')

## Decryption

In [145]:
# (U, c, r)

In [146]:
T_prime = apply_double_and_add_method( G = U, k = ka, p = p)

In [147]:
assert T_prime == T

In [148]:
k1_prime , k2_prime = derive_keys(T_prime)

In [149]:
r_prime = find_mac(message= c, key = k2_prime)

In [150]:
assert r_prime == r 

In [152]:
obj_alice = AES.new(k1_prime, AES.MODE_CBC)
plaintext = obj_alice.decrypt(c)

In [153]:
plaintext

b'\xe6b\xe6\x0c\xcc\xa30\x98\x90\x83\xdd l\xfd\x80\x95'